In [2]:
pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.9/648.9 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 38.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as bs
import json
import pymongo
import logging

In [7]:
logging.basicConfig(filename="scrapper.log" , level=logging.INFO)

In [8]:
# setting up url
searchString = 'study table'
flipkart_url = "https://www.flipkart.com/search?q=" + searchString.replace(" ", "%20")

In [9]:
# Open the url, read it into a a local variable and close the connection 

logging.info(f"Trying URL Request : {flipkart_url}")
try:
    uClient = uReq(flipkart_url)
except Exception as e:
    logging.info(f"URL Request : {flipkart_url} - Failed : {uClient}")
else:
    search_page = uClient.read()
    uClient.close()
    logging.info(f"URL Request : {flipkart_url} - Read and closed succesfully")

In [10]:
# Parse the search page results into html parser
search_page_html = bs(search_page, "html.parser")

# All individual items are under div class = _4ddWXP
search_items = search_page_html.find_all("div", {"class" : "_4ddWXP"})
# return [] if len(search_items) = 0
logging.info(f"{len(search_items)} items found for {searchString}")

# initialize a list to store all the item details with reviews
products_reviews = []
# print(len(search_items))

# loop through the search items and access individual item titel and url
for item in search_items:
    item_dict = {}
    
    try:
        item_dict['title'] = item.find_all("a", {"class" : "s1Q9rs"})[0]['title']
    except:
        logging.info(item_dict['title'])
    
    try:
        item_dict['href'] = item.find_all("a", {"class" : "s1Q9rs"})[0]['href']
    except:
        logging.info(item_dict['href'])
        
    
    # Goto the item URL
    item_url = "https://www.flipkart.com" + item_dict['href']
    try:
        logging.info(f"Trying URL Request : {item_url}")
        itemUCLient = uReq(item_url)
    except:
        logging.info(item_url)
    else:
        item_page = itemUCLient.read()
        itemUCLient.close()
        logging.info(f"URL Request : {item_url} - Read and closed succesfully")
        
    item_page_html = bs(item_page, "html.parser")
    
    try:
        original_price = item_page_html.find("div", {"class" : "_3I9_wc _2p6lqe"}).text
    except:
        logging.info(original_price)
        
    try:
        selling_price = item_page_html.find("div", {"class" : "_30jeq3 _16Jk6d"}).text
    except:
        logging.info(selling_price)
    
    
    
    item_dict['price'] = {"original_price" : original_price, "selling_price" : selling_price}
    
    review_box = item_page_html.find_all("div", {"class" : "_16PBlm"})
    logging.info(f"{len(review_box)} reviews found for product - {item_dict['title']}")
    # print(f"{len(review_box)} reviews")
    # print(review_box[0])
    
    reviews = []
    for user_review in review_box:
        try:
            user = user_review.find("p", {"class" : "_2sc7ZR _2V5EHH"}).text
        except:
            user
        try:
            rating = user_review.find("div", {"class" : "_3LWZlK _1BLPMq"}).text
        except:
            rating
        try:
            title = user_review.find("p", {"class" : "_2-N8zT"}).text
        except:
            title
        try:
            review = user_review.find("div", {"class" : "t-ZTKy"}).div.text
        except:
            review

        user_review_dict = {"user" : user, "rating" : rating, "title" : title, "review" : review}
        reviews.append(user_review_dict)
    
    item_dict['reviews'] = reviews
    products_reviews.append(item_dict)

json_file = searchString.replace(" ", "_") + "flipkart_search" + ".json"
with open(json_file, "w") as json_file:
    json.dump(products_reviews, json_file)

    

